In [1]:
# https://www.pyimagesearch.com/2017/07/17/credit-card-ocr-with-opencv-and-python/
from imutils import contours
import numpy as np
import imutils
import cv2

In [2]:
# map the first credit card number to the credit card type
FIRST_NUMBER = {
    "3": "American Express",
    "4": "Visa",
    "5": "MasterCard",
    "6": "Discover Card"
}

In [20]:
def disp_image(image, title="image"):
    cv2.startWindowThread()
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [21]:
# load the reference OCR-A image from disk, convert it to grayscale,
# and threshold it, such that the digits appear as "white" on a 
# "black" background and invert it
ref = cv2.imread("ocr_a_digits.png")
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
disp_image(ref)
ref = cv2.threshold(ref, 180, 255, cv2.THRESH_BINARY)[1]
disp_image(ref)

In [15]:
# find countours in the OCR-A image
# sort them from left to right, and initialize a dictionary to map
# digit name to the ROI
refCnts = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL,
                    cv2.CHAIN_APPROX_SIMPLE)
refCnts = refCnts[0] if imutils.is_cv2() else refCnts[1]
refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
digits = {}

In [17]:
# loop over the OCR-A reference contours
for (i, c) in enumerate(refCnts):
    # compute the bounding box for the digit, extract it, and resize
    # it to a fixed size
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ref[y:y+h, x:x+w]
    roi = cv2.resize(roi, (57, 88))
    
    digits[i] = roi

In [19]:
# initialize a rectangular (wider than it is tall) and
# square structuring kernel
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
SqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

In [27]:
# load the input image, resize it, and convert it to grayscale
image = cv2.imread("images/card1.jpg")
image = imutils.resize(image, width=300)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
disp_image(gray)

In [28]:
# apply a tophat (whitehat) morphological operator to find light
# regions against a dark background (i.e., the credit card numbers)
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel)
disp_image(tophat)